In [4]:
# %pip install tqdm
# %pip install python-dotenv
# %pip install torch==2.4.0+cu118
# %pip install scikit_learn==1.2.2
# %pip install ipython
# %pip install pandas
# %pip install numpy
# %pip install matplotlib
# %pip install tabulate
# %pip install scipy
# %pip install git+https://github.com/Louis-Li-dev/ML_tool_kit

In [5]:
import os
import sys
parent_dir = os.path.join(os.getcwd(), '..')
if parent_dir not in sys.path: sys.path.append(parent_dir)
from utility.data_utils import *
from utility.visuals import *
from dotenv import load_dotenv
from model.CNN import ConditionalSegmentationVAE
from mkit.torch_support.tensor_utils import xy_to_tensordataset
from torch import nn
from IPython.display import clear_output
from sklearn.ensemble import RandomForestRegressor
load_dotenv()
DATA_DIR = os.getenv("DATA_DIR")

- Dataset

In [6]:
if not os.path.exists(DATA_DIR): raise FileNotFoundError("Make sure the data directory is correctly placed.")

In [7]:
files = get_files(DATA_DIR)

return_list = []
file = files[0]
city_name = file.split('\\')[-1].split('.csv')[0].split('_')[0]

path_name = process_and_transform_data(file, resolution=.5, overwrite=True)
with open(path_name, 'rb') as f:
    result_dict = pickle.load(f)
labels = result_dict['labels']
encoder = result_dict['encoder']
MAX_LEN = result_dict['max length']
file_name = result_dict['file name']
WIDTH = result_dict['width']
HEIGHT = result_dict['height']

original dataset size: 238
dataset size with duplicates removed: 172


In [8]:
# for idx, label in enumerate(labels):
#     plt.imshow(labels[idx])
#     plt.savefig(f'../fig/{idx}_{file_name}.png')
#     plt.show()


- x y splitting

In [9]:
from sklearn.model_selection import train_test_split


unique_labels = [u for u in labels if np.array(np.where(u != 0)).T.shape[0] > 1]
padded_labels = []
for label in unique_labels:
    unique_vals = np.unique(label)[1:]
    new_vals = []
    count = 0
    for val in unique_vals:    
        dummy_vals = np.zeros(label.shape)
        dummy_vals[np.where(label == val)] = 1
        new_vals.append(dummy_vals)
        count += 1
    for i in range(count, MAX_LEN):
        dummy_vals = np.zeros(label.shape)
        new_vals.append(dummy_vals)
    new_vals = np.array(new_vals)
    padded_labels.append(new_vals)
train_labels, test_labels = train_test_split(padded_labels, test_size=.2)


In [10]:
import torch
import torch.nn as nn

class Encoder(nn.Module):
    def __init__(self, input_dim, start_dim, n_layers):
        super(Encoder, self).__init__()
        layers = [
            nn.Conv2d(input_dim, start_dim, kernel_size=3, stride=2, padding=1),
            nn.InstanceNorm2d(start_dim),
            nn.Mish(),
        ]
        
        in_channels = start_dim
        for _ in range(n_layers):
            out_channels = in_channels * 2
            layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=2, padding=1))
            if _ != n_layers - 1:
                layers.append(nn.InstanceNorm2d(out_channels))
            layers.append(nn.Mish())
            in_channels = out_channels
        
        self.encoder = nn.Sequential(*layers)

    def forward(self, x):
        return self.encoder(x)

class Decoder(nn.Module):
    def __init__(self, start_dim, n_layers, output_dim):
        super(Decoder, self).__init__()
        in_channels = start_dim * (2 ** n_layers)
        layers = []
        
        for _ in range(n_layers):
            out_channels = in_channels // 2
            layers.append(nn.ConvTranspose2d(in_channels, out_channels, kernel_size=3, stride=2, padding=1, output_padding=1))
            layers.append(nn.InstanceNorm2d(out_channels))
            layers.append(nn.Mish())
            in_channels = out_channels
        
        layers.append(nn.ConvTranspose2d(in_channels, output_dim, kernel_size=3, stride=2, padding=1, output_padding=1))
        layers.append(nn.Sigmoid())  # Ensure output is between 0 and 1 for images
        
        self.decoder = nn.Sequential(*layers)

    def forward(self, x):
        return self.decoder(x)

class Autoencoder(nn.Module):
    def __init__(self, input_dim, start_dim, n_layers, output_dim):
        super(Autoencoder, self).__init__()
        self.encoder = Encoder(input_dim, start_dim, n_layers)
        self.decoder = Decoder(start_dim, n_layers, output_dim)

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

def generate(input_dim, start_dim, n_layers, output_dim, device="cpu", output_type="autoencoder"):
    """
    Creates an encoder, decoder, or autoencoder model based on user input.

    Args:
        input_dim (int): Number of input channels (e.g., 3 for RGB).
        start_dim (int): The first convolution layer's channel count.
        n_layers (int): Number of encoder layers (each doubling start_dim).
        output_dim (int): The number of output channels.
        device (str): 'cpu' or 'cuda' (GPU).
        output_type (str): 'encoder', 'decoder', or 'autoencoder'.

    Returns:
        A PyTorch model on the selected device.
    """
    device = torch.device(device)

    if output_type == "encoder":
        model = Encoder(input_dim, start_dim, n_layers)
    elif output_type == "decoder":
        model = Decoder(start_dim, n_layers, output_dim)
    elif output_type == "autoencoder":
        model = Autoencoder(input_dim, start_dim, n_layers, output_dim)
    else:
        raise ValueError("Invalid output_type. Choose from 'encoder', 'decoder', or 'autoencoder'.")
    return model.to(device)


In [11]:
import torch
import torch.nn as nn

class Encoder(nn.Module):
    def __init__(self, input_dim, start_dim, n_layers, latent_dim, img_width, img_height):
        super(Encoder, self).__init__()
        layers = [
            nn.Conv2d(input_dim, start_dim, kernel_size=3, stride=1, padding=1),
            nn.InstanceNorm2d(start_dim),
            nn.Mish(),
        ]
        
        in_channels = start_dim
        for _ in range(n_layers):
            out_channels = in_channels * 2
            layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1))
            if _ != n_layers - 1:
                layers.append(nn.InstanceNorm2d(out_channels))
            layers.append(nn.Mish())
            in_channels = out_channels

        self.encoder = nn.Sequential(*layers)

        # Compute the final spatial size after convolutions
        self.final_width = img_width
        self.final_height = img_height 
        self.final_channels = in_channels

        # Fully Connected Layers for Latent Space
        self.fc_mu = nn.Linear(self.final_channels * self.final_width * self.final_height, latent_dim)
        self.fc_logvar = nn.Linear(self.final_channels * self.final_width * self.final_height, latent_dim)

    def forward(self, x):
        x = self.encoder(x)
        
        x = x.view(x.size(0), -1)  # Flatten
        
        mu = self.fc_mu(x)
        logvar = self.fc_logvar(x)
        return mu, logvar

class Decoder(nn.Module):
    def __init__(self, start_dim, n_layers, output_dim, latent_dim, img_width, img_height):
        super(Decoder, self).__init__()
        
        # Compute the final spatial size
        self.final_width = img_width
        self.final_height = img_height
        self.final_channels = start_dim * (2 ** n_layers)

        # Fully Connected Layer to reshape back to feature map
        self.fc = nn.Linear(latent_dim, self.final_channels * self.final_width * self.final_height)

        layers = []
        in_channels = self.final_channels
        for _ in range(n_layers):
            out_channels = in_channels // 2
            layers.append(nn.ConvTranspose2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1))
            layers.append(nn.InstanceNorm2d(out_channels))
            layers.append(nn.Mish())
            in_channels = out_channels
        
        layers.append(nn.ConvTranspose2d(in_channels, output_dim, kernel_size=3, stride=1, padding=1))
        
        self.decoder = nn.Sequential(*layers)

    def forward(self, z):
        x = self.fc(z)
        x = x.view(x.size(0), self.final_channels, self.final_width, self.final_height)  # Reshape to feature map
        return self.decoder(x)

class VAE(nn.Module):
    def __init__(self, input_dim, start_dim, n_layers, output_dim, latent_dim, img_width, img_height):
        super(VAE, self).__init__()
        self.encoder = Encoder(input_dim, start_dim, n_layers, latent_dim, img_width, img_height)
        self.decoder = Decoder(start_dim, n_layers, output_dim, latent_dim, img_width, img_height)

    def reparameterize(self, mu, logvar):
        """
        Reparameterization trick: sample z ~ N(mu, sigma^2)
        """
        std = torch.exp(0.5 * logvar)  # Convert log variance to std
        eps = torch.randn_like(std)  # Sample from standard normal
        return mu + eps * std  # Reparameterized latent vector

    def forward(self, x):
        mu, logvar = self.encoder(x)
        z = self.reparameterize(mu, logvar)
        reconstructed = self.decoder(z)
        return reconstructed, mu, logvar

def vae_loss(recon_x, x, mu, logvar):
    """
    Compute the VAE loss function:
    Loss = Reconstruction Loss + KL Divergence
    """
    recon_loss = nn.functional.l1_loss(recon_x, x, reduction='sum')  # BCE Loss
    kl_div = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())  # KL Divergence Loss
    return recon_loss + kl_div

def generate(input_dim, start_dim, n_layers, output_dim, latent_dim, img_width, img_height, device="cpu", output_type="vae"):
    """
    Creates an encoder, decoder, or VAE model based on user input.

    Args:
        input_dim (int): Number of input channels (e.g., 3 for RGB).
        start_dim (int): The first convolution layer's channel count.
        n_layers (int): Number of encoder layers (each doubling start_dim).
        output_dim (int): The number of output channels.
        latent_dim (int): Size of the latent representation.
        img_width (int): Input image width.
        img_height (int): Input image height.
        device (str): 'cpu' or 'cuda' (GPU).
        output_type (str): 'encoder', 'decoder', or 'vae'.

    Returns:
        A PyTorch model on the selected device.
    """
    device = torch.device(device)

    if output_type == "encoder":
        model = Encoder(input_dim, start_dim, n_layers, latent_dim, img_width, img_height)
    elif output_type == "decoder":
        model = Decoder(start_dim, n_layers, output_dim, latent_dim, img_width, img_height)
    elif output_type == "vae":
        model = VAE(input_dim, start_dim, n_layers, output_dim, latent_dim, img_width, img_height)
    else:
        raise ValueError("Invalid output_type. Choose from 'encoder', 'decoder', or 'vae'.")
    return model.to(device)


- data processing

In [12]:

loader, val_loader = xy_to_tensordataset(
    train_labels, train_labels,
    return_loader=True, 
    batch_size=8,
    input_dtype=torch.float32,
    output_dtype=torch.float32,
    val_ratio=.15,
)

c:\Users\ss348\anaconda3\Lib\site-packages\mkit\torch_support\tensor_utils.py:438: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:281.)
  return torch.tensor(arr)


In [13]:
import torch.optim as optim

def train_vae(model, train_loader, optimizer, device="cuda"):
    model.train()
    total_loss = 0

    for batch_idx, (x, _) in enumerate(train_loader):  # Assume data is (images, labels)
        x = x.to(device)

        optimizer.zero_grad()
        recon_x, mu, logvar = model(x)

        loss = vae_loss(recon_x, x, mu, logvar)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        if batch_idx % 100 == 0:
            print(f"Batch {batch_idx}/{len(train_loader)}, Loss: {loss.item()}")

    avg_loss = total_loss / len(train_loader.dataset)
    return avg_loss

def validate_vae(model, val_loader, device="cuda"):
    model.eval()
    total_loss = 0

    with torch.no_grad():
        for x, _ in val_loader:
            x = x.to(device)
            recon_x, mu, logvar = model(x)

            loss = vae_loss(recon_x, x, mu, logvar)
            total_loss += loss.item()

    avg_loss = total_loss / len(val_loader.dataset)
    return avg_loss
device = "cuda" if torch.cuda.is_available() else "cpu"

vae = generate(input_dim=MAX_LEN, start_dim=32, n_layers=2, output_dim=MAX_LEN, latent_dim=128, img_width=WIDTH, img_height=HEIGHT, device=device)
optimizer = optim.Adam(vae.parameters(), lr=1e-4)

num_epochs = 100

for epoch in tqdm(range(num_epochs)):
    train_loss = train_vae(vae, loader, optimizer, device)
    val_loss = validate_vae(vae, val_loader, device)

    print(f"Epoch [{epoch+1}/{num_epochs}] -> Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")


  0%|          | 0/100 [00:00<?, ?it/s]

Batch 0/15, Loss: 110989.9453125


  1%|          | 1/100 [00:02<04:17,  2.60s/it]

Epoch [1/100] -> Train Loss: 11029.3370, Val Loss: 6662.4688
Batch 0/15, Loss: 53010.921875


  2%|▏         | 2/100 [00:04<03:12,  1.96s/it]

...

Epoch [98/100] -> Train Loss: 573.6072, Val Loss: 578.1544
Batch 0/15, Loss: 4678.8603515625


 99%|█████████▉| 99/100 [02:31<00:01,  1.53s/it]

Epoch [99/100] -> Train Loss: 569.1872, Val Loss: 574.5159
Batch 0/15, Loss: 4580.98095703125


100%|██████████| 100/100 [02:33<00:00,  1.53s/it]

Epoch [100/100] -> Train Loss: 570.1748, Val Loss: 579.0814


In [14]:
pred = vae(torch.tensor(train_labels, device=torch.device('cuda')).float())

In [ ]:
plt.imshow(train_labels[1][3])

In [ ]:

pred_np = pred[0].cpu().detach().numpy()
count = 0
for ele in pred_np:
    print(ele[0])
    plt.imshow(ele[0], vmax=1, vmin=0)
    count += 1
    plt.savefig('../fig/model_figs/' + str(count) + '.png')
      

[[ 0.00530585  0.00633608 -0.00638314 ...  0.00518536 -0.00131768
   0.00907136]
 [ 0.00221615  0.00256875  0.00782794 ... -0.00094632 -0.01038616
  -0.01108993]
 [ 0.00177421 -0.00223001  0.01357553 ... -0.01358774 -0.00376088
   0.00159471]
 ...
 [ 0.00295518 -0.01272563 -0.00072261 ...  0.00638906 -0.00952257
  -0.01091492]
 [ 0.00035631 -0.00960466  0.00403384 ...  0.00229139  0.00248394
  -0.00643252]
 [ 0.00429464  0.02110469 -0.00134287 ... -0.00969957 -0.00710573
   0.00029883]]
[[-0.00072498  0.01165395 -0.01203468 ... -0.00026623 -0.00352166
  -0.00156105]
 [-0.01218804  0.00872565  0.0049587  ...  0.01449727  0.00385981
  -0.01701123]
 [-0.00077201  0.0016752   0.0153474  ... -0.00931995 -0.00846378
   0.00990718]
 ...
 [ 0.01136915 -0.01241709 -0.00327679 ... -0.00210019 -0.0082287
  -0.02145121]
 [-0.00609333 -0.00463604 -0.00419081 ... -0.01975758  0.00138683
  -0.00873357]
 [ 0.00126963  0.01228953 -0.01093542 ...  0.004294   -0.01047825
  -0.00312002]]
[[-0.00735565  0.

In [ ]:
count = np.array(np.where(mask.cpu().detach().numpy() != False)).T[:, 0]
with open('xxx.txt', 'w') as f:
    f.write(str(count.tolist()))


NameError: name 'mask' is not defined

In [ ]:
plt.imshow(train_labels[15][0])

In [ ]:

mask = pred
masked_pred = pred * mask
plt.imshow(pred[15][0].cpu().detach())


- For Machine Learning Models
    - To fit the data formats of tensors, every sci-kit learn model needs to be wrapped inside the object MLWrapper

In [ ]:
class MLWrapper(nn.Module):
    def __init__(self, model_object = RandomForestRegressor, **args):
        self.model = model_object(**args)
        self.device = torch.device('cpu')
    def loader_to_xy(self, loader):
        x, y = loader.dataset.tensors
        x, y = np.array(x).squeeze(1), np.array(y)
        x = x.reshape(x.shape[0], -1)
        return x, y
    def fit(self, train_loader, val_loader):
        train_x, train_y = self.loader_to_xy(train_loader)
        val_train_x, val_train_y = self.loader_to_xy(val_loader)
        self.model.fit(train_x, train_y)
        accu = self.model.score(val_train_x, val_train_y)
        print(accu)
    def inference(self, img):
        batch_size, _, _, _ = img.shape
        img = img.reshape(batch_size, -1)
        return torch.tensor(self.model.predict(img))

- VAE

In [ ]:
IMG_CHANNELS = 1    # For grayscale images; use 3 for RGB.
N_EPOCHS = 100      # Adjust as needed.
LATENT_DIM = 300    # Dimensionality of the latent space.
FEATURE_MAPS = 8    # Base number of feature maps.

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ConditionalSegmentationVAE(
    latent_dim=LATENT_DIM,
    width=WIDTH,
    height=HEIGHT,
    img_channels=IMG_CHANNELS,
    feature_maps=FEATURE_MAPS,
    device=device
)

segmentation_loss_fn = nn.BCEWithLogitsLoss().to(device)

# Assume your train_loader and val_loader are defined appropriately.
model.train_vae(
    train_loader=loader,       # your training DataLoader
    val_loader=val_loader,       # your validation DataLoader
    n_epochs=N_EPOCHS,
    seg_criterion=segmentation_loss_fn,
    kl_weight=0.001,
    patience=10,
    device=device
)
clear_output(wait=True)

In [ ]:

evaluate_and_plot(test_loader, model=model, encoder=encoder, title='VAE', dataset_name=city_name)

- Random Forest

In [ ]:


model = MLWrapper()
model.fit(loader, val_loader)
evaluate_and_plot(test_loader, model=model, encoder=encoder, title='RF', dataset_name=city_name)